In [1]:
import swarmrl.engine.resobee as resobee
import subprocess
import multiprocessing
import os
infomsg = "I "

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
Could not find espressomd. Features will not be available


In [2]:
resobee_root_path = "/home/ac141173/repos/swarm-rc/"
build_path = os.path.join(resobee_root_path, "build")
target = 'many_body_simulation'


In [3]:
from IPython.utils import io
with io.capture_output() as captured:

    # run this to rebuild the resobee executable
    max_workers = multiprocessing.cpu_count()
    print(infomsg, f"Found {max_workers} cores.")

    # print(infomsg)
    # print(infomsg, "Clean previous build...")
    # cmd = ['cmake', '--build', build_path, '--target', 'clean']
    # print(infomsg, ' '.join(cmd))
    # subprocess.run(cmd)

    # print(infomsg)
    # print(infomsg, "Configuring cmake...")
    # cmd = ['cmake', f'-B {build_path}']
    # print(infomsg, ' '.join(cmd))
    # subprocess.run(cmd)

    print(infomsg)
    print(infomsg, f"Building target {target}...")
    cmd = ["cmake", "--build", build_path, "--target", target, "-j", str(max_workers)]
    print(infomsg, ' '.join(cmd))
    subprocess.run(cmd)

[  4%] Call shell script for writing git hash
[  4%] Built target write_git_hash
[ 52%] Built target yaml-cpp
Consolidate compiler generated dependencies of target many_body_simulation
[ 52%] Building CXX object src/CMakeFiles/many_body_simulation.dir/mainSimulation.cpp.o
[ 52%] Building CXX object src/CMakeFiles/many_body_simulation.dir/simulation/Simulation.cpp.o
[ 52%] Building CXX object src/CMakeFiles/many_body_simulation.dir/simulation/interactions/forces/swarmRL/SwarmRLAction.cpp.o


In file included from /home/ac141173/repos/swarm-rc/src/simulation/interactions/forces/swarmRL/SwarmRLAction.cpp:1:
/home/ac141173/repos/swarm-rc/src/simulation/interactions/forces/swarmRL/SwarmRLAction.h: In member function ‘virtual void SwarmRLAction::operator()(ForcePopulation&)’:
/home/ac141173/repos/swarm-rc/src/simulation/interactions/forces/swarmRL/SwarmRLAction.h:54:24: warning: ignoring return value of ‘zmq::recv_result_t zmq::detail::socket_base::recv(zmq::message_t&, zmq::recv_flags)’, declared with attribute ‘nodiscard’ [-Wunused-result]
   54 |             socket.recv(reply, zmq::recv_flags::none);
      |             ~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
In file included from /home/ac141173/repos/swarm-rc/src/simulation/interactions/forces/swarmRL/SwarmRLAction.h:9,
                 from /home/ac141173/repos/swarm-rc/src/simulation/interactions/forces/swarmRL/SwarmRLAction.cpp:1:
/home/ac141173/repos/swarm-rc/build/_deps/cppzmq-src/zmq.hpp:2036:19: note: declared here

[ 56%] Linking CXX executable many_body_simulation
[100%] Built target many_body_simulation


In [4]:
system_runner = resobee.ResoBee(
    resobee_executable=os.path.join(resobee_root_path, 'build/src', target),
    config_dir=os.path.join(resobee_root_path, 'workflow/projects/debug/parameter-combination-0/seed-0')
)

Binding to tcp address:  tcp://127.0.0.1:5543


In [5]:
system_runner.integrate(0, None)


Listening for requests from ResoBee client...
The Many-Body Simulations and Machine Learning Group proudly presents...
▒▒░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░██  ░░██  ░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░  
▒▒  ░░  ██░░░░░░░░░░░░░░░░░░░░░░  ░░    ██      ░░░░░░░░░░░░░░░░░░░░░░░░    ██
▓▓▓▓    ▒▒░░░░░░░░░░░░░░░░░░░░░░    ██▓▓▓▓░░  ▒▒░░░░░░░░░░░░░░░░░░░░░░▒▒  ░░▓▓
░░▒▒▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▓▓  ░░░░▓▓▒▒░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░
▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▓▓░░░░██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░▓▓░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░░░████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒████░░░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▒▒████▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓████▒▒░░░░░░░░░░░░░░
░░░░░░░░░░░░░░░░▓▓▒▒████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▓▓██▓▓▓▓░░░░░░░░░░░░░░
░░░░░░░░░░░░░░▒▒░░░░██  ▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒  ██▒▒░░▒▒░░░░░░░░░░░░
░░░░░░░░░░░░▒▒  ░░  ██░░  ▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░  ▒▒░░░░░░░░░░
░░░░░░░░░░██